In [1]:
pip install --upgrade transformers

In [2]:
!pip install sentencepiece

In [3]:
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [4]:
!pip install torch torchvision torchaudio

In [5]:
import torch
print(torch.__version__)

2.3.1+cu121


In [6]:
pip show sentencepiece

Name: sentencepiece
Version: 0.1.99
Summary: SentencePiece python wrapper
Home-page: https://github.com/google/sentencepiece
Author: Taku Kudo
Author-email: taku@google.com
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: 


In [7]:
import sentencepiece
print("SentencePiece imported successfully.")

SentencePiece imported successfully.


In [8]:
from transformers import T5Tokenizer
try:
    tokenizer=T5Tokenizer.from_pretrained('t5-small')
    print("T5Tokenizer loaded successfully.")
except Exception as e:
    print(f"Error: {e}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


T5Tokenizer loaded successfully.


In [9]:
#How2sign  dataset
import json
import os.path

import numpy as np #importing necessary libraries
import pandas as pd


class How2signDataset:
    def __init__(self, json_files, csv_file, seq_len=183, time_len=512):
        self.seq_len = seq_len
        self.time_len = time_len
        self.json_files = json_files
        self.csv_file = csv_file

        self.sentence_dict = self.load_y()

    def get_x(self, x_path):
        # Load the JSON data
        with open(x_path, 'r') as f:
            data = json.load(f)

        # Extract the hand_pose_face keypoints
        hand_pose_faces = [person['hand_pose_face'] for person in data['people']]

        hand_pose_faces = np.array(hand_pose_faces)
        hand_pose_faces = hand_pose_faces.reshape(1, -1, self.seq_len)

        # Create an array to store the padded data
        x = np.zeros((1, self.time_len, self.seq_len))

        # Fill the padded array with the actual data
        seq_length = len(hand_pose_faces[0])
        x[:, :seq_length, :] = hand_pose_faces[:, :seq_length, :]

        return x

    def load_y(self):
        data = pd.read_csv(self.csv_file, delimiter='\t', on_bad_lines='skip')
        df = data[['SENTENCE_NAME', 'SENTENCE']]
        sentence_dict = pd.Series(df.SENTENCE.values, index=df.SENTENCE_NAME).to_dict()

        return sentence_dict

    def get_y(self, x_base_path):
        y = self.sentence_dict.get(x_base_path, "0")

        return y

    def how2sign_keypoints_sentence(self):
        # Load the data from multiple files
        x = [self.get_x(json_file) for json_file in self.json_files]
        x = np.concatenate(x, axis=0)

        json_files_base = [json_file.split(".")[0] for json_file in self.json_files]
        print(json_files_base)
        y = [self.get_y(json_file_base) for json_file_base in json_files_base]
        y = np.array(y)

        # Concatenate the data from the files
        return x, y

In [11]:
json_files = ["/content/drive/MyDrive/CO6qyvvglAE_18-5-rgb_front.json"]
csv_file = "/content/drive/MyDrive/how2sign_realigned_val.csv"

# json_files = find_files("data/how2sign/realigned_val", pattern='**/*.json', interval=1)
dataset = How2signDataset(json_files=json_files, csv_file=csv_file)
x, y = dataset.how2sign_keypoints_sentence()

print(x)
print(y)
print('-'*100)
print(x.shape)
print(y.shape)

['/content/drive/MyDrive/CO6qyvvglAE_18-5-rgb_front']
[[[0.64002734 0.63365875 0.639195   ... 0.5492125  0.30881625 0.887098  ]
  [0.64035156 0.62237875 0.580242   ... 0.55052266 0.30683125 0.922378  ]
  [0.63711953 0.59722625 0.608995   ... 0.55191484 0.30503375 0.947329  ]
  ...
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]]]
['0']
----------------------------------------------------------------------------------------------------
(1, 512, 183)
(1,)


In [12]:
from torch import nn


def pair(t):
    return t if isinstance(t, tuple) else (t, t)


class PatchEmbedding(nn.Module):
    def __init__(self, patch_dim, max_frames, dim, drop_prob=0.1, device=None):
        """
        Args:
            max_frames (int): number of frames
            dim (int): embedding dimension
        Returns:
            nn.Embedding: embedding layer shaped as (batch, num_patches, dim)
        """
        super().__init__()
        self.device = device

        self.to_patch_embedding = nn.Sequential(
            nn.LayerNorm(patch_dim),
            nn.Linear(patch_dim, dim),
            nn.LayerNorm(dim),
        )

    def forward(self, x):
        return self.to_patch_embedding(x)


In [14]:
import os
import json
from transformers import T5Tokenizer
from torch.utils.data import Dataset, DataLoader
from glob import glob

import json
import os

def  find_files(directory, pattern='**/*.json'):
    return glob(os.path.join(directory,pattern),recursive=True)

'''# Define paths to your folders
train_folder = "/content/drive/MyDrive/train_2D_keypoints"
val_folder = "/content/drive/MyDrive/val_2D_keypoints"
test_folder =  "/content/drive/MyDrive/test_2D_keypoints" '''

'''# Load the datasets
train_data = find_files(train_folder)
val_data = find_files(val_folder)
test_data = find_files(test_folder)'''



# Tokenizer and Dataset Preparation
tokenizer = T5Tokenizer.from_pretrained('t5-small')



'''class How2signDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        inputs = self.tokenizer.encode_plus(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze()
        } '''

# Create datasets and dataloaders
#y data
# In 'ipython-input-17-68a0c3077465' file, uncomment the following lines:
train_DATA= "//content/drive/MyDrive/train_2D_keypoints"
csv_file = "/content/drive/MyDrive/how2sign_realigned_train.csv"
train_data=find_files(train_DATA)
train_dataset = How2signDataset(train_data,csv_file) # Create an instance of the training dataset
# x_train, y_train = train_data.how2sign_keypoints_sentence()
# print(x_train.shape, y_train.shape)

val_DATA= "/content/drive/MyDrive/val_2D_keypoints"
csv_file = "/content/drive/MyDrive/how2sign_realigned_val.csv"
val_data=find_files(val_DATA)
# val_dataset = How2signDataset(val_data, csv_file)
# x_val, y_val = val_data.how2sign_keypoints_sentence()
# print(x_val.shape, y_val.shape)


test_DATA = "/content/drive/MyDrive/test_2D_keypoints" # Verify this path
csv_file = "/content/drive/MyDrive/how2sign_realigned_test.csv"
test_data=find_files(test_DATA)
# Check if test_data is empty and print a message if it is
if not test_data:
    print("No JSON files found in the directory. Please verify the path.")
else:
    test_dataset = How2signDataset(test_data,csv_file)
    x_test, y_test = test_dataset.how2sign_keypoints_sentence()
    print(x_test.shape, y_test.shape)



#train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
#val_dataloader = DataLoader(val_dataset, batch_size=8)
test_dataloader = DataLoader(test_dataset, batch_size=8)


['/content/drive/MyDrive/test_2D_keypoints/openpose_output/output/-g0iPSnQt6w_4-1-rgb_front', '/content/drive/MyDrive/test_2D_keypoints/openpose_output/output/fzQ2E3_xBZA_2-5-rgb_front', '/content/drive/MyDrive/test_2D_keypoints/openpose_output/output/fzQ2E3_xBZA_2-8-rgb_front', '/content/drive/MyDrive/test_2D_keypoints/openpose_output/output/FZCF7kPIyOk_6-1-rgb_front', '/content/drive/MyDrive/test_2D_keypoints/openpose_output/output/fzQ2E3_xBZA_3-8-rgb_front', '/content/drive/MyDrive/test_2D_keypoints/openpose_output/output/fzQ2E3_xBZA_3-5-rgb_front', '/content/drive/MyDrive/test_2D_keypoints/openpose_output/output/-g0iPSnQt6w_6-1-rgb_front', '/content/drive/MyDrive/test_2D_keypoints/openpose_output/output/FZCF7kPIyOk_7-1-rgb_front', '/content/drive/MyDrive/test_2D_keypoints/openpose_output/output/fZM3IcM2Xs4_2-5-rgb_front', '/content/drive/MyDrive/test_2D_keypoints/openpose_output/output/-g0iPSnQt6w_7-1-rgb_front', '/content/drive/MyDrive/test_2D_keypoints/openpose_output/output/fZM3

In [15]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

patch_dim=183
max_frames=512
dim=1
class T5WithProjection(nn.Module):
    def __init__(self, t5_model_name, embed_dim):
        super(T5WithProjection, self).__init__()
        self.t5 = T5ForConditionalGeneration.from_pretrained('t5-small')
        self.projection = PatchEmbedding(patch_dim,max_frames,dim)

    def forward(self, input_ids, attention_mask=None, labels=None):
         # Apply linear projection
        input_ids = self.projection(outputs.logits)  # Apply projection to logits
        # Forward pass through T5 model
        outputs = self.t5(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        return outputs

# Initialize the model with projection
embed_dim = 1  # Adjust according to your T5 model's hidden size
model = T5WithProjection('t5-small', embed_dim)


In [16]:
import torch
from transformers import AdamW, T5ForConditionalGeneration, T5Tokenizer
from tqdm import tqdm
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

# Define the How2signDataset class
class How2signDataset(Dataset):
    def __init__(self, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.max_len = max_len
        # Load your dataset here. This is just a placeholder.
        self.data = [
            {"input_text": "Translate this text to sign language.", "target_text": "some_translation"},
            # Add more data samples
        ]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        input_text = item['input_text']
        target_text = item['target_text']

        # Tokenize the input and target text
        inputs = self.tokenizer.encode_plus(
            input_text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        targets = self.tokenizer.encode_plus(
            target_text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].squeeze(),  # Remove extra dimension
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': targets['input_ids'].squeeze()  # Use input_ids as labels
        }

# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('t5-small')
model.to(device)
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Initialize the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Define your dataset and dataloader
max_len = 512
train_dataset = How2signDataset(tokenizer, max_len)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Define the training function
def train_epoch(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

# Train the model
for epoch in range(3):  # Number of epochs
    train_loss = train_epoch(model, train_dataloader, optimizer, device)
    print(f"Epoch {epoch + 1}: Training loss = {train_loss:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1: Training loss = 7.4362
Epoch 2: Training loss = 9.5622
Epoch 3: Training loss = 8.0334


In [17]:
# Save the model
model.save_pretrained('t5-with-projection')
tokenizer.save_pretrained('t5-with-projection')

('t5-with-projection/tokenizer_config.json',
 't5-with-projection/special_tokens_map.json',
 't5-with-projection/spiece.model',
 't5-with-projection/added_tokens.json')